In [1]:
import gym
import tensorflow as tf
import numpy as np
import random

In [2]:
env = gym.make('CartPole-v1')

In [3]:
#param
l = 0.9 # lambda
a = 0.1 # learning rate
eg = 1.0 # epsilon greedy
egMin = 0.05

In [4]:
class Agent(tf.keras.Model):

  def __init__(self):
    super(Agent, self).__init__(name='mon_agent')
    # Define your layers here.
    self.dense = tf.keras.layers.Dense(2, activation='relu', input_shape=(4,))

  def call(self, inputs):
    return self.dense(inputs)


In [5]:
class MaskedMSE(tf.keras.losses.Loss) :
  def call(self, y_true, y_pred):
    print( y_true, y_pred)
    y_true = y_true * y_pred["mask"]
    y_pred = y_pred["mask"] * y_pred["value"]
    #ops.convert_to_tensor(y_pred)
    r = tf.keras.losses.mse(y_true, y_pred)
    print(y_true, y_pred, r)
    return r
    #y_pred = ops.convert_to_tensor(y_pred)
    #y_true = math_ops.cast(y_true, y_pred.dtype)
    #return K.mean(math_ops.square(y_pred - y_true), axis=-1)


In [6]:
agent = Agent()
agent.compile(optimizer=tf.keras.optimizers.Adam(),
#              loss=MaskedMSE,
    metrics=['accuracy'])

agent.build(tf.TensorShape([None,4]))
#encoder.build((5,1,1))
agent.summary()

#print(agent([[0,0,0,0]]));


Model: "mon_agent"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [7]:
#i = 0
#r = 0

for epi in range(1,50) :
    memory = { "states":[], "targets":[]}
    state = env.reset()

    # * * * explore * * *
    for step in range(epi * 100):
        env.render()
        #print(state)
        if (random.random() > eg) :
            Qs = agent([state])[0];
            action = np.argmax(Qs);
            Q = Qs[action]
            #print("State: ", state, " Qs: ", Qs.numpy(), " Action: ", action, " Q: ", Q.numpy())
        else :
            Qs = agent([state])[0];
            action = random.randrange(2)
            Q = Qs[action]
            #print("State: ", state, " Action: ", action, " Q: ", Q.numpy())
            
        memory["states"].append(state)
        mask = [0,0]
        mask[action] = 1
        state, rewards, done, info = env.step(action)
        
        Qs = agent([state])[0];
        Q = Q + a * (rewards + l * np.max(Qs) - Q)
        #print("rewards: ", rewards, " Qs: ", Qs.numpy(), " Q: ", Q.numpy(), " mask: ", mask)
        
        memory["targets"].append( { "mask": mask, "action":action, "value":Q } )
        if done: state = env.reset()
        
    #print(memory)
    eg = max(egMin, eg*.95)
    print("eG: ", eg)
    
    # * * * learn * * *
#    agent.fit(memory["states"], memory["targets"], batch_size=32, steps_per_epoch=10)
    break
    
    
    

eG:  0.95


In [8]:
print(memory["states"], memory["targets"])

[array([ 0.02749484, -0.04465773, -0.01172934, -0.00055651]), array([ 0.02660169, -0.23960952, -0.01174047,  0.28840271]), array([ 0.0218095 , -0.4345621 , -0.00597242,  0.5773598 ]), array([ 0.01311825, -0.62959983,  0.00557478,  0.8681553 ]), array([ 5.26257315e-04, -8.24797190e-01,  2.29378855e-02,  1.16258577e+00]), array([-0.01596969, -0.62998134,  0.0461896 ,  0.87718201]), array([-0.02856931, -0.43551655,  0.06373324,  0.59937085]), array([-0.03727964, -0.24134146,  0.07572066,  0.32742445]), array([-0.04210647, -0.04737462,  0.08226915,  0.05954849]), array([-0.04305397, -0.24357388,  0.08346012,  0.37701187]), array([-0.04792544, -0.4397759 ,  0.09100035,  0.69479928]), array([-0.05672096, -0.63603434,  0.10489634,  1.01468607]), array([-0.06944165, -0.83238697,  0.12519006,  1.3383791 ]), array([-0.08608939, -1.02884331,  0.15195764,  1.66746721]), array([-0.10666625, -1.22537103,  0.18530699,  2.00336562]), array([ 0.04812939, -0.04125079,  0.01056953, -0.02583695]), array([

In [9]:
env.close()

In [46]:
#print(memory["states"][0], memory["targets"][0], memory["targets"][0]["value"].numpy())
#x = [[-0.01512732,  0.02205488, -0.02799687,  0.00528442]]
#y = [0.14108522 , 0.14108522]
#agent.fit(x, y, batch_size=1, steps_per_epoch=10)
#.fit(memory["states"], memory["targets"], batch_size=32, steps_per_epoch=10)

class LossTest(tf.keras.losses.Loss) :
    def call(self, y_true, y_pred):
        #print(y_pred, y_true)
        r = tf.keras.metrics.mean_squared_error(y_pred, y_true)
        return r

class Trainning(tf.keras.Model):
    def __init__(self, agent):
        super(Trainning, self).__init__()
        # Define your layers here.
        self.agent = agent
        #self.dense = tf.keras.layers.Dense(2, activation='relu', input_shape=(4,))

    def call(self, inputs):
        [states, masks] = inputs
        #tf.print(states)
        #tf.print(masks)
        #tf.keras.backend.print_tensor(states)
        #tf.keras.backend.print_tensor(masks)
        #print(inputs)
        #print(states, masks)
        return agent(states) * mask
        
        #return self.dense(inputs)


agent = Agent()
agent.compile(optimizer=tf.keras.optimizers.Adam(),
    loss=LossTest(),
#    sample_weight_mode="temporal",
    metrics=['accuracy'])

agent.build(tf.TensorShape([None,4]))

trainning = Trainning(agent)
trainning.compile(optimizer=tf.keras.optimizers.Adam(),
    loss=LossTest(),
#    sample_weight_mode="temporal",
    metrics=['accuracy'])

#trainning.build([tf.TensorShape([None,4]), tf.TensorShape([None,2])])
state_input = tf.keras.Input(shape=(4), name='state_input')
mask_input = tf.keras.Input(shape=(2), name='mask_input')
trainning([state_input, mask_input])
#tf.keras.utils.plot_model(trainning, 'multi_input_and_output_model.png', show_shapes=True)


#trainning.build(tf.TensorShape([[None,4], [None,2]]))
#trainning.build([None,4])

#agent.summary()

data = np.random.random((10, 4))

masks = np.array([ [0.,1.] if random.random()<=0.5 else [1.,0.] for i in range(10) ])
#masks = np.zeros((10,2))
labels = np.random.random((10, 2)) * masks
#for i in masks :
#    print(i)
#print(labels);
#print(type(masks), masks);
#masks = np.zeros((10,2))
#print(type(masks), masks);

#print(masks)
#print(data, labels)
#agent.fit(data, labels, sample_weight=masks ,epochs=10, batch_size=32)
#agent.fit(data, labels, epochs=10, batch_size=32)
his = trainning.fit([data, masks], labels, epochs=10, batch_size=10)
#print (his)
#print(agent.input_spec)

Train on 10 samples
Epoch 1/10
10/10 [==============================] - 0s 4ms/sample - loss: 0.2139 - accuracy: 0.6000
Epoch 2/10
10/10 [==============================] - 0s 199us/sample - loss: 0.2138 - accuracy: 0.6000
Epoch 3/10
10/10 [==============================] - 0s 100us/sample - loss: 0.2137 - accuracy: 0.6000
Epoch 4/10
10/10 [==============================] - 0s 100us/sample - loss: 0.2136 - accuracy: 0.6000
Epoch 5/10
10/10 [==============================] - 0s 199us/sample - loss: 0.2135 - accuracy: 0.6000
Epoch 6/10
10/10 [==============================] - 0s 199us/sample - loss: 0.2134 - accuracy: 0.6000
Epoch 7/10
10/10 [==============================] - 0s 100us/sample - loss: 0.2133 - accuracy: 0.6000
Epoch 8/10
10/10 [==============================] - 0s 200us/sample - loss: 0.2133 - accuracy: 0.6000
Epoch 9/10
10/10 [==============================] - 0s 299us/sample - loss: 0.2132 - accuracy: 0.6000
Epoch 10/10
10/10 [==============================] - 0s 199us/sa